# Discriminator network alone
This network distinguishes real color images and images which were colored by our network. After teaching it we can use it to initialize a GAN.

In [2]:
from keras.layers import Input, Reshape, Dropout, BatchNormalization, Conv2D, UpSampling2D, MaxPooling2D, concatenate
from keras.layers.core import Activation, Dense, Flatten
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras.models import load_model, save_model
from matplotlib.colors import LinearSegmentedColormap
from skimage.color import rgb2lab, lab2rgb
import matplotlib.pyplot as plt
import numpy as np
import h5py
from keras.utils.generic_utils import get_custom_objects
import tensorflow as tf
import csv, os
from PIL import Image as PImage
from IPython.display import clear_output
from keras.callbacks import CSVLogger

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
# yields a batch for discriminator neural neetwork
# X_batch: images with all 3 lab channels, the first half is predicted by generator
#          and the second half is the real images
# Y_batch: 0 - if fake, 1 - if real
def generatorDisc(h5dataset, generator, batch_half_size = 32, seed = 1):
    indices = list(range(h5dataset.shape[0])) # array of the indices
    while True: # infinite generator
        np.random.seed(seed) # set seed so shuffle will be repeatable
        np.random.shuffle(indices) # shuffle before every epoch
        for i in range(0, len(indices) - batch_half_size + 1, batch_half_size):
            X_batch = np.zeros((2* batch_half_size,) + h5dataset.shape[1:]) # create space for X_batch
            Y_batch = np.array([[0] * batch_half_size, [1] * batch_half_size]).flatten() # create Y_batch
            # load pictures and normalize them
            batch_indices = sorted(indices[i:i+batch_half_size]) # only sorted list can index more elements at once
            lab_batch = h5dataset[batch_indices].astype(np.float32) # get Lab pictures from dataset
            lab_batch[:,:,:,0:1] = lab_batch[:,:,:,0:1]/100 # scale inputs (L layer)
            lab_batch[:,:,:,1:3] = (lab_batch[:,:,:,1:3] +128) / 255 # scale outputs (A and B layers)
            
            # load X_bacth with values
            X_batch[0:batch_half_size] = generator.predict(lab_batch[:,:,:,0:1]) # predicted
            X_batch[batch_half_size:] = lab_batch # real

            yield (X_batch,Y_batch)
            
# plots the original pictures, the predicted ones by model, and the inputs
def plot_results(model, picture):
    if np.max(picture) > 1.01:
        picture = picture.astype(np.float32)
        picture[:,:,:,0:1] = picture[:,:,:,0:1]/100 # L channel
        picture[:,:,:,1:3] = (picture[:,:,:,1:3] + 128)/255 # A and B channels
    preds = model.predict(picture[:,:,:,0:1]) # predictin A and B channels
#     preds = np.concatenate((picture[:,:,:,0:1],preds),axis=3) # adding the L channel (input) to the picture
    plt.rcParams["figure.figsize"] = [15,10] # bigger plot size
    
    for ind in range(picture.shape[0]):
        plt.subplot(131)
        plt.title('Original')
        plt.grid(False) # no grid
        plt.axis('off') # no axes
        plt.imshow(lab2rgb(outputscaler(picture[ind])))

        plt.subplot(132)
        plt.title('Predicted')
        plt.grid(False) # no grid
        plt.axis('off') # no axes
        plt.imshow(lab2rgb(outputscaler(preds[ind])))

        plt.subplot(133)
        plt.title('Black&White')
        plt.grid(False) # no grid
        plt.axis('off') # no axes
        cmL = LinearSegmentedColormap.from_list('L channel', [(0, 0, 0), (1, 1, 1)], N=101)
        plt.imshow(picture[ind,:,:,0], cmap=cmL, vmin = 0, vmax = 1)
        plt.show()

def outputscaler(img): #function to upscale our image for the excel output, the plt.imshow() finction does not need upscaling
    #funtcion of min-max scaling: y = (x-min(x))/(max(x)-min(x)), we need to reverse this: x = y*(max(x)-min(x)) + min(x)
    ret = np.zeros(img.shape)
    ret[:,:,0] = np.round(img[:,:,0]*100)     #L channel goes from 0 - +100
    ret[:,:,1] = np.round((img[:,:,1]*255))-128 #a channel goes from -128 to +127
    ret[:,:,2] = np.round((img[:,:,2]*255))-128 #b channel goes from -128 to +127
    return ret

In [4]:
# loads the last model with the given name
# saved models should follow the naming method of 'CheckPointer'
# net_name: name of the net
def loadLastModel(net_name):
    net_name = net_name.upper() # using uppercases, so net_name is not case sensitive
    # list of previous saved networks
    previous_nets = [x for x in os.listdir('weights/' + net_name + '/') if net_name in x]
    if len(previous_nets) == 0: # if there were no previous
        # print some feedback
        print('ERROR: There is no previous models with this name:', net_name)
    else: # if we have at least one previous model
        # print some feedback
        print('Loading model:', previous_nets[-1])
        # return with the loaded model
        # path: 'weights/NETNAME/NETNAME_XX.hdf5'
        return load_model('weights/' + net_name + '/' + previous_nets[-1])

# Create network

In [4]:
inputs = Input((128,128,3))
        
conv1 = Conv2D(16, (3,3), padding = 'same')(inputs)
conv1 = BatchNormalization(axis = 3, momentum = 0.9)(conv1)
conv1 = Activation('relu')(conv1)

conv2 = Conv2D(32, (3, 3), padding='same', strides=2)(conv1)
conv2 = BatchNormalization(axis = 3, momentum = 0.9)(conv2)
conv2 = Activation('relu')(conv2)

conv3 = Conv2D(64, (3, 3), padding='same', strides=2)(conv2)
conv3 = BatchNormalization(axis = 3, momentum = 0.9)(conv3)
conv3 = Activation('relu')(conv3)

conv4 = Conv2D(128, (3, 3), padding='same', strides=2)(conv3)
conv4= BatchNormalization(axis = 3, momentum = 0.9)(conv4)
conv4 = Activation('relu')(conv4)

conv5 = Conv2D(256, (3, 3), padding='same', strides=2)(conv4)
conv5 = BatchNormalization(axis = 3, momentum = 0.9)(conv5)
conv5 = Activation('relu')(conv5)

maxpool = MaxPooling2D((2,2))(conv5)
maxpool = MaxPooling2D((2,2))(maxpool)
maxpool = MaxPooling2D((2,2))(maxpool)

flat = Flatten()(maxpool)

dense1 = Dense(256)(flat)
dense1 = Activation('relu')(dense1)

dense2 = Dense(128)(dense1)
dense2 = Activation('relu')(dense2)

dense3 = Dense(1)(dense2)
dense3 = Activation('sigmoid')(dense3)

disc1 = Model(inputs, dense3)
disc1.name = 'discriminator'
print('Discriminator:')
disc1.summary()

disc1.compile(loss = 'binary_crossentropy',
                      optimizer = 'adam',
                      metrics=['accuracy'])

Discriminator:
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 128, 128, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 128, 16)      448       
_________________________________________________________________
batch_normalization_1 (Batch (None, 128, 128, 16)      64        
_________________________________________________________________
activation_1 (Activation)    (None, 128, 128, 16)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 64, 32)        4640      
_________________________________________________________________
batch_normalization_2 (Batch (None, 64, 64, 32)        128       
_________________________________________________________________
activation_2 (Activation)    (None, 64, 64, 32)        0     

In [7]:
h5Lab = h5py.File('data/allDataLab.h5') # loading dataset

# Loading generator network
We used UNET (Unet4_72.hdf5) as the generator.

In [4]:
# custom loss function
def SSIMLoss(y_true, y_pred):
    # SSIM gives higher number for better similarity, so we need the opposite of it
    return 1 - tf.image.ssim(y_true, y_pred, 1)
# add loss function to keras
get_custom_objects().update({"SSIMLoss": SSIMLoss})

# loading trained UNET
generator = loadLastModel('unet4')
# adding the input layer to the output layers
inputs = generator.input
outputs = concatenate([inputs, generator.output], axis = 3, name = 'concat')
generator = Model(inputs, outputs)

generator.summary()

Loading model: UNET4_72.hdf5
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 128, 128, 1)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 128, 128, 32) 320         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 128, 128, 32) 128         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 128, 128, 32) 0           batch_normalization_1[0][0]      
________________________________________________________________________________

# Teaching the network
We used UNET (UNET4_72.hdf5) as generator and teached the discriminator for 2 epochs.

In [13]:
train_gen = generatorDisc(h5Lab['train'], generator, batch_half_size = 128) # train data generator
valid_gen = generatorDisc(h5Lab['valid'], generator, batch_half_size = 128) # validation data generator

In [14]:
disc1.fit_generator(train_gen, steps_per_epoch = h5Lab['train'].shape[0]//128, epochs = 10,
                    validation_data = valid_gen, validation_steps = h5Lab['valid'].shape[0]//128,
                    callbacks = [CSVLogger('logs/disc1.csv', append = True, separator = ',')])

Epoch 1/10
1319/1319 [==============================] - 1299s 985ms/step - loss: 0.0315 - acc: 0.9862 - val_loss: 0.1658 - val_acc: 0.9395
Epoch 2/10
1319/1319 [==============================] - 1302s 987ms/step - loss: 0.0013 - acc: 0.9995 - val_loss: 2.4663e-04 - val_acc: 0.9999
Epoch 3/10
  62/1319 [>.............................] - ETA: 15:53 - loss: 0.0014 - acc: 0.9996

KeyboardInterrupt: 

# Test the network

In [16]:
# testing discriminator

test_gen = generatorDisc(h5Lab['test'], generator, batch_half_size = 128) # test data generator

tn = 0 # true negative
tp = 0 # true positive

steps = h5Lab['test'].shape[0]//128 # steps to scan whole test data
count = steps*128 # number of test data per category (false/real)

for i in range(steps):
    batch = next(test_gen) # new batch
    preds = disc1.predict(batch[0]) # predicts
    tn += sum([1 for x in preds[0:128] if x < 0.5]) # counting true negative
    tp += sum([1 for x in preds[128:] if x > 0.5]) # counting true positive
print('False acc:', 100*tn/count, '\nReal acc:', 100*tp/count) # print results

False acc: 99.58444148936171 
Real acc: 99.97506648936171


In [18]:
# the testing gave pretty good results so save the model
save_model(disc1,'weights/DISC1/DISC1_01.hdf5')